# LayerLens ImageNet Demo

This notebook demonstrates how to use LayerLens to explain a pre-trained ImageNet model (ResNet50).

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# Import LayerLens
import sys
sys.path.append('..')  # Add parent directory to path
import layerlens as ll

## 1. Load a pre-trained ResNet50 model

In [ ]:
# Load the pre-trained model
model = ResNet50(weights='imagenet')

# Display model summary
print("Model loaded: ResNet50")
print(f"Number of layers: {len(model.layers)}")

## 2. Load and preprocess an image

In [ ]:
def load_and_preprocess_image(img_path):
    """Load and preprocess an image for ResNet50."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array), img

# You can replace this with any image path
img_path = tf.keras.utils.get_file(
    "elephant.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/elephant.jpg"
)

# Load and preprocess the image
processed_img, original_img = load_and_preprocess_image(img_path)

# Display the original image
plt.figure(figsize=(6, 6))
plt.imshow(original_img)
plt.axis('off')
plt.title("Original Image")
plt.show()

## 3. Make predictions with the model

In [ ]:
# Make predictions
preds = model.predict(processed_img)

# Decode and display the top 5 predictions
decoded_preds = decode_predictions(preds, top=5)[0]
print("Top 5 predictions:")
for i, (imagenet_id, label, score) in enumerate(decoded_preds):
    print(f"{i+1}: {label} ({score:.4f})")

# Set the top prediction
top_pred_idx = np.argmax(preds[0])
top_pred_label = decoded_preds[0][1]
print(f"\nTop prediction: {top_pred_label} (index {top_pred_idx})")

## 4. Use LayerLens to explain the model

In [ ]:
# Create a LayerLens explainer
# We'll focus on a subset of layers to speed up the explanation process
target_layers = [
    'conv1_conv', 
    'conv2_block3_out', 
    'conv3_block4_out', 
    'conv4_block6_out', 
    'conv5_block3_out',
    'avg_pool',
    'predictions'
]

explainer = ll.Explainer(model, layers=target_layers)

# Generate explanations
print("Generating explanations (this may take a while)...")
explanations = explainer.explain(processed_img)

print(f"Generated explanations for {len(explanations.layer_explanations)} layers")

## 5. Visualize layer activations

In [ ]:
# Extract layer outputs
from layerlens.core.layer_extractor import LayerExtractor

extractor = LayerExtractor(model, layers=target_layers)
layer_outputs = extractor.extract(processed_img)

# Visualize activations for an intermediate layer
from layerlens.visualization.heatmap_generator import generate_heatmap

# Select a convolutional layer to visualize
conv_layer = 'conv3_block4_out'
conv_activations = layer_outputs[conv_layer]

# Generate and display the heatmap
conv_heatmap = generate_heatmap(conv_activations, conv_layer, processed_img[0], overlay=True)

# Display the heatmap
from plotly.offline import iplot
iplot(conv_heatmap)

## 6. Compare activations across multiple layers

In [ ]:
# Compare activations across different layers
from layerlens.visualization.heatmap_generator import compare_layer_activations

# Select conv layers to compare
conv_layers = ['conv1_conv', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']
conv_activations = {layer: layer_outputs[layer] for layer in conv_layers}

# Generate the comparison visualization
comparison_fig = compare_layer_activations(conv_activations, processed_img[0])

# Display the comparison
iplot(comparison_fig)

## 7. Analyze feature importance for the final prediction

In [ ]:
# Build a surrogate model for the final layer
from layerlens.core.surrogate_builder import SurrogateBuilder

# We need more samples to build a reliable surrogate
# For the demo, we'll use random noise as additional samples
n_samples = 20
random_samples = np.random.randn(n_samples, 224, 224, 3)
random_samples = preprocess_input(random_samples)

# Combine with our real sample
all_samples = np.concatenate([processed_img, random_samples], axis=0)

# Extract features from the penultimate layer
extractor = LayerExtractor(model, layers=['avg_pool', 'predictions'])
penultimate_outputs = extractor.extract(all_samples)

# Build surrogate for the final layer
surrogate_builder = SurrogateBuilder(surrogate_type='linear')
final_surrogate = surrogate_builder.fit(
    'predictions', 
    penultimate_outputs['avg_pool'].reshape(n_samples + 1, -1), 
    penultimate_outputs['predictions']
)

# Get feature importances for the top prediction class
if hasattr(final_surrogate, 'coef_'):
    # For linear models, we can look at the coefficients
    coefs = final_surrogate.coef_[top_pred_idx]
    
    # Plot the top feature importances
    from layerlens.utils.plot_utils import plot_feature_importance
    
    importance_fig = plot_feature_importance(
        np.abs(coefs),  # Use absolute value of coefficients
        feature_names=None,
        top_n=20
    )
    plt.show()

## 8. Visualize the model graph

In [ ]:
# Visualize the model architecture (showing only our target layers)
from layerlens.visualization.layer_graph import plot_layer_graph

# Create a submodel with only the layers we're analyzing
submodel = tf.keras.Model(
    inputs=model.input,
    outputs=[model.get_layer(layer).output for layer in target_layers]
)

# Create the graph visualization
layer_graph = plot_layer_graph(submodel, highlight_layers='predictions')

# Display the graph
iplot(layer_graph)

## 9. Generate a feature flow visualization

In [ ]:
# Visualize how features flow through the network
from layerlens.visualization.feature_flow import plot_feature_flow

# Create the feature flow visualization
feature_flow_fig = plot_feature_flow(explanations, processed_img)

# Display the visualization
iplot(feature_flow_fig)

## 10. Compare explanations for different images

In [ ]:
# Load a second image for comparison
img_path2 = tf.keras.utils.get_file(
    "tiger.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/tiger.jpg"
)

# Load and preprocess the image
processed_img2, original_img2 = load_and_preprocess_image(img_path2)

# Display the second image
plt.figure(figsize=(6, 6))
plt.imshow(original_img2)
plt.axis('off')
plt.title("Second Image")
plt.show()

# Make predictions
preds2 = model.predict(processed_img2)
decoded_preds2 = decode_predictions(preds2, top=3)[0]
print("Top 3 predictions for second image:")
for i, (imagenet_id, label, score) in enumerate(decoded_preds2):
    print(f"{i+1}: {label} ({score:.4f})")

# Generate explanations for the second image
explanations2 = explainer.explain(processed_img2)

# Compare feature flows
from layerlens.visualization.feature_flow import compare_feature_flows

# Create a comparison of feature flows
comparison_fig = compare_feature_flows(
    explanations,
    [processed_img[0], processed_img2[0]],
    sample_labels=["Elephant", "Tiger"]
)

# Display the comparison
iplot(comparison_fig)

## Conclusion

In this notebook, we've demonstrated how to use LayerLens to explain a pre-trained ResNet50 model on ImageNet data. We've:

1. Extracted and visualized activations from different layers
2. Built surrogate models to explain the final classification layer
3. Analyzed feature importance for specific predictions
4. Compared explanations across different images

LayerLens helps us understand how complex deep learning models process images and arrive at their final predictions by providing layer-by-layer insights.